In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from glob import glob
import cv2
from scipy.ndimage import label as measure_label
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt

from ink_helpers import (load_image,seed_everything,
                         load_fragment,)

In [3]:
def get_landmark_height(mask3D_binary):
    landmark_height = np.zeros(mask3D_binary.shape[1:], dtype=np.uint8)
    for layer_idx in range(mask3D_binary.shape[0]):
        landmark_height[mask3D_binary[layer_idx] > 0] = layer_idx
    return landmark_height

In [ ]:
all_frag_ids = ['1', '2a', '2b', '2c', '3']
id2dir = {id:f'./frags/train_{id}' for id in all_frag_ids}

id2images,id2frag_mask,id2ink_mask,id2papyrus_mask = {},{},{},{}
for frag_id in tqdm(all_frag_ids):
    images,frag_mask,ink_mask = load_fragment(frag_id)
    id2images[frag_id] = images
    id2frag_mask[frag_id] = frag_mask
    id2ink_mask[frag_id] = ink_mask
    id2papyrus_mask[frag_id] = np.load(f'{id2dir[frag_id]}/quick_mask3D_binary.npy')

 80%|████████████████████████████████████         | 4/5 [01:43<00:26, 26.29s/it]

In [ ]:
num_top_channel = 17
num_bottom_channel = 18
kernel_size = 101

# create landscape surface mask
for frag_id in tqdm(id2images):
    
    # load image and fragment masks
    frag_mask = id2frag_mask[frag_id]
    images = id2images[frag_id]
    papyrus_mask = id2papyrus_mask[frag_id]

    landmark_height = get_landmark_height(papyrus_mask)
    
    # generate a smooth height map
    papyrus_mean_height = int(round(landmark_height[landmark_height>0].mean()))
    landmark_height[landmark_height==0] = max(num_bottom_channel, papyrus_mean_height)
    landmark_height[landmark_height<num_bottom_channel] = num_bottom_channel
    landmark_height = gaussian_filter(landmark_height, kernel_size)
    landmark_height[landmark_height>papyrus_mask.shape[0]-num_top_channel] = papyrus_mask.shape[0]-num_top_channel
    
    surface_arr = np.zeros(shape=(num_top_channel+num_bottom_channel, *frag_mask.shape), dtype=np.uint8)
    for pos,relative_h in enumerate(range(-num_bottom_channel, num_top_channel)):
        
        expanded_index = np.expand_dims(landmark_height+relative_h, axis=0)
        surface_arr[pos, :, :] = np.take_along_axis(images, expanded_index, axis=0)
    surface_arr[surface_arr==0] = int(images[-1][frag_mask].mean())
        
    np.save(f'{id2dir[frag_id]}/landscape_peel_surface_kernel{kernel_size}_top{num_top_channel}_bottom{num_bottom_channel}.npy', 
            surface_arr, )

In [ ]:
papyrus_mask.shape

In [ ]:
plt.imshow(landmark_height, cmap='Set2')
plt.colorbar()